# Exercise 1
* Train and fine-tune a decision tree for the moons dataset by following these steps:
    * Use make_moons(n_samples=10000, noise=0.4) to generate a moons dataset.
    * Use train_test_split() to split the dataset into a training set and a test set.
    * Use grid search with cross-validation (with the help of the GridSearchCV class) to find good hyperparameter values for a DecisionTreeClassifier. Hint: try various values for max_leaf_nodes.
    * Train it on the full training set using these hyperparameters, and measure your model’s performance on the test set. You should get roughly 85% to 87% accuracy.

## Importing Libraries

In [2]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [9]:
## create the moons dataset
X_moons, y_moons = make_moons(n_samples=10000, noise=0.4, random_state=42)

In [10]:
## reading the features
X_moons

array([[ 0.9402914 ,  0.12230559],
       [ 0.12454026, -0.42477546],
       [ 0.26198823,  0.50841438],
       ...,
       [-0.24177973,  0.20957199],
       [ 0.90679645,  0.54958215],
       [ 2.08837082, -0.05050728]])

In [11]:
y_moons

array([1, 0, 0, ..., 1, 0, 1])

Observations:
* So we have 2 features in this dataset, lets callthem $x_1$ and $x_2$, and our target variable is $y$